In [1]:
import pandas as pd
import string
import re
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [36]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence


In [2]:
df_test = pd.read_csv('../raw_data/titles_data/titles_test.csv')
df_train = pd.read_csv('../raw_data/titles_data/titles_train.csv')


In [28]:
df_test

,Image_name,Title
0,044310073X.jpg,Sereed Camion Oral and Maxillofacial Surgery A...
1,1438005687.jpg,BARRON'S The Leader in Test Preparation GRE MO...
2,0060750715.jpg,The Ballet Maker wwww. George Balanchine Rober...
3,1580237959.jpg,INSTITUTE FOR JEWISH SPIRITUALITY SERIES A PAR...
4,0135137829.jpg,Tu Oce P and P and Co acred Construction Sched...
...,...,...
5695,0399174680.jpg,#1 NEW YORK TIMES BESTSELLING AUTHOR STUART WO...
5696,1939454654.jpg,STORIES FROM THE AGE OF DISTRACTION VOLUME ONE...
5697,1604601604.jpg,Fifth Moved in the Culasidge Community Series ...
5698,0399166726.jpg,WX -LIBA Saving Jason MICHAEL SEARS BLACK FRIDAYS


In [24]:
def preprocessed_title(titles):
    output = titles.str.lower()
    output = output.apply(lambda x: "".join([i for i in x if i not in string.punctuation]))
    output = output.apply(lambda x: "".join([i for i in x if i not in string.digits]))
    output = output.apply(lambda x: re.split('\W+', x))
    #output = output.apply(lambda x: [str(TextBlob(word).correct()) for word in x])
    porter_stemmer = PorterStemmer()
    output = output.apply(lambda x: [porter_stemmer.stem(word) for word in x])
    stops = set(stopwords.words('english'))
    output = output.apply(lambda x: [word for word in x if word not in stops])

    return output

In [27]:
X_train_preproc = preprocessed_title(df_train['Title'].fillna(''))

In [29]:
X_test_preproc = preprocessed_title(df_test['Title'].fillna(''))

In [37]:
X_train_preproc

0        [colet, exampl, sa, sport, ronal, gentl, twitt...
1                      [maghet, pull, togeth, push, apart]
2        [energi, secur, volum, sage, librari, intern, ...
3        [audiobook, mpcd, insert, transfer, listen, tr...
4                [citi, rock, idaho, climber, guid, untal]
                               ...                        
51295    [john, sandford, new, york, timesbestsel, auth...
51296    [second, edit, crime, writer, guid, polic, pra...
51297               [jezebel, remedi, nord, martin, clark]
51298                              [startup, glenn, ogura]
51299    [bad, karaden, mysteri, mysteri, ender, killer...
Name: Title, Length: 51300, dtype: object

## Word2vec - pretrained

In [31]:
!pip freeze | grep gensim

gensim==4.2.0


In [32]:
!pip freeze | grep tensorflow

tensorflow==2.13.0rc1
tensorflow-datasets==4.6.0
tensorflow-estimator==2.13.0rc0
tensorflow-macos==2.13.0rc1
tensorflow-metadata==1.10.0


In [64]:
#X_train_embed = [text_to_word_sequence(str(title)) for title in X_train_preproc]

AttributeError: 'list' object has no attribute 'split'

In [65]:
len(X_train_preproc)

51300

In [95]:
X_train_preproc

0        [colet, exampl, sa, sport, ronal, gentl, twitt...
1                      [maghet, pull, togeth, push, apart]
2        [energi, secur, volum, sage, librari, intern, ...
3        [audiobook, mpcd, insert, transfer, listen, tr...
4                [citi, rock, idaho, climber, guid, untal]
                               ...                        
51295    [john, sandford, new, york, timesbestsel, auth...
51296    [second, edit, crime, writer, guid, polic, pra...
51297               [jezebel, remedi, nord, martin, clark]
51298                              [startup, glenn, ogura]
51299    [bad, karaden, mysteri, mysteri, ender, killer...
Name: Title, Length: 51300, dtype: object

In [110]:
word2vec = Word2Vec(sentences=X_train_preproc, min_count=10, vector_size=20, window=3) 
#wv = word2vec.wv

In [111]:
print(word2vec.wv['leadership'])


[-0.17825377  0.48358867  0.27253163 -0.19101048 -0.9312971  -0.15243877
 -0.29092497  0.5761052  -0.6898378   0.08620487  0.83036035  0.42400408
  0.47332236 -0.68770665 -0.01304691  0.16790017  0.35935917 -0.11217155
 -0.917654   -0.7011325 ]


In [112]:
word2vec.wv.most_similar('crime')

[('origin', 0.9635210633277893),
 ('fiction', 0.946500837802887),
 ('short', 0.9432647228240967),
 ('fantasi', 0.9432392716407776),
 ('ancient', 0.9120355844497681),
 ('detect', 0.9112628102302551),
 ('major', 0.9039660692214966),
 ('christian', 0.9032929539680481),
 ('western', 0.9029826521873474),
 ('prison', 0.9023937582969666)]

In [99]:
word2vec.wv.vectors.shape

(7275, 20)